Copyright 2023-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Nonstationary tutorial

This notebook demonstrates how to use the specialized lensing shear kernel (hard-coded to RBF at the moment).

⚠️ _Note that this is still an experimental feature._ ⚠️

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from MuyGPyS._src.gp.tensors import _pairwise_differences
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.distortion import IsotropicDistortion, l2, F2
from MuyGPyS.gp.hyperparameter import ScalarHyperparameter
from MuyGPyS.gp.hyperparameter.experimental import (
    sample_knots,
    HierarchicalNonstationaryHyperparameter,
)
from MuyGPyS.gp.kernels import RBF
from MuyGPyS.gp.noise import HomoscedasticNoise

This is required to import the implementation from Bob Armstrong's original repository.
It must be cloned in the same directory as MuyGPyS for the relative paths to work.

In [ ]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("analytic_kernel", "../../shear_kernel/analytic_kernel.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["analytic_kernel"] = foo
spec.loader.exec_module(foo)
from analytic_kernel import (
    kernelf as kernelf_o,
    shear_kernel as shear_kernel_o,
)

We also import the in-library baseline implementation for sanity checking.

In [ ]:
from MuyGPyS._test.shear import shear_kernel as shear_kernel_n

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

Here we build some simple data, which is mean to represent a grid of sky coordinates.

In [ ]:
n = 25  # number of galaxies on a side
xmin = 0
xmax = 1
ymin = 0
ymax = 1

xx = np.linspace(xmin, xmax, n)
yy = np.linspace(ymin, ymax, n)

x, y = np.meshgrid(xx, yy)
features = np.vstack((x.flatten(), y.flatten())).T

Here we construct a shear value kernel (partial differential components of RBF), as well as the original RBF kernel using Bob's implementation.

In [ ]:
def original_workflow():
    # shear value kernel
    vals = np.zeros((3 * (n) ** 2, 3 * (n) ** 2))
    # original kernel
    ovals = np.zeros(((n) ** 2, (n) ** 2))
    vals[:] = np.nan
    ovals[:] = np.nan
    for i, (ix, iy) in enumerate(features):
        for j, (jx, jy) in enumerate(features):
            vals[i * 3 : (i + 1) * 3, j * 3 : (j + 1) * 3] = shear_kernel_o(ix, iy, jx, jy)
            ovals[i, j] = kernelf_o(ix, iy, jx, jy)
    return vals, ovals

In [ ]:
vals_o, ovals_o = original_workflow()

Here we do the same using the MuyGPyS implementation. Note the increased efficiency.

In [ ]:
def MuyGPyS_workflow():
    diffs = _pairwise_differences(features)
    # shear value kernel
    vals = shear_kernel_n(diffs)
    # original kernel
    kernelf_n = RBF(metric=IsotropicDistortion(metric=F2, length_scale=ScalarHyperparameter(1.0)))
    ovals = kernelf_n(diffs)
    return vals, ovals

In [ ]:
vals_n, ovals_n = MuyGPyS_workflow()

Do the two implementations agree?

In [ ]:
np.all((np.allclose(vals_o, vals_n), np.allclose(ovals_o, ovals_n)))

Plot results of the baseline and internal implementations. 

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8))
axes[0, 0].imshow(vals_o)
axes[0, 0].set_title("original shear kernel")
axes[0, 1].imshow(ovals_o)
axes[0, 1].set_title("original rbf kernel")
axes[1, 0].imshow(vals_n)
axes[1, 0].set_title("MuyGPyS shear kernel")
axes[1, 1].imshow(ovals_n)
axes[1, 1].set_title("MuyGPyS rbf kernel")
plt.show()

Runtime comparison of the two implementations:

In [ ]:
if True:
    %timeit original_workflow()
    %timeit MuyGPyS_workflow()